In [9]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import accuracy_score
import csv
import cv2 as cv
import numpy as np
from pathlib import Path
import sys
print(Path.cwd())
sys.path.insert(0, str(Path.cwd() / "image_normalization"))
from image_norm import setGrayToBlack, paddImage
from pymage_size import get_image_size
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/Users/julieschult/Desktop/BiomedicalSignalProcessing/TTT23_Project


2023-11-06 13:11:24.395797: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def dataaug(img):
    datagen = ImageDataGenerator(
        rotation_range=40,  # Randomly rotate images by up to 40 degrees
        zoom_range=0.3,  # Zoom in/out randomly by up to 20%
        horizontal_flip=True,  # Flip images horizontally
        brightness_range=[0.5, 1.5]  # Adjust brightness randomly in the range [0.5, 1.5]
    )
    image = cv2.imread('rawdata/EXO_6_16.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.reshape((1,) + image.shape)
    # Generate augmented images
    augmented_images = []
    for batch in datagen.flow(image, batch_size=1):
        augmented_images.append(batch[0])
        if len(augmented_images) >= 3:  # Generate 3 augmented images
            break

In [10]:
def readImages(csvPath):

    filenames = []
    roi = []
    max_shape = [0, 0]
    with open(csvPath) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                path = Path.cwd() / row[0]
                filenames.append(str(path))
                img_size = get_image_size(str(path)).get_dimensions()[::-1]
                if img_size[0] > max_shape[0]:
                    max_shape[0] = img_size[0]
                if img_size[1] > max_shape[1]:
                    max_shape[1] = img_size[1]

                roi.append([float(row[1]), float(row[2]), float(row[3])])

    y = np.zeros((len(filenames), 3), float)
    images = np.zeros((len(filenames), max_shape[0], max_shape[1]), dtype=np.uint8) 
    for i, filepath in enumerate(filenames):
        img = cv.imread(str(filepath))
        # cv.imshow("img1", img)
        img_black = setGrayToBlack(img, threshold=150)
        # cv.imshow("img_gray_black", img)

        img_gray = cv.cvtColor(img_black, cv.COLOR_BGR2GRAY)
        # cv.imshow("img_gray", img)
        cv.imwrite('test.jpg', img_gray)
        img_pad = paddImage(img_gray, max_shape)
        # cv.imshow("pad", img)
        images[i] = img_pad
        y[i] = np.array(roi[i])
    
    return images, y



def normalizeY(y):
    return (y-np.amin(y))/(np.amax(y) - np.amin(y))

In [11]:
images, y = readImages(r"Rawdata_values.csv")
y = normalizeY(y)
X = images.reshape((images.shape[0], images.shape[1]*images.shape[2])) # Flatten images

Column names are ﻿path, ROI1, ROI2, ROI3


In [4]:
def mse(y_pred, y_test):
    MSEs = [0, 0, 0]
    n = len(y_pred)
    for idx, i in enumerate(y_pred):
        for idx2, j in enumerate(i):
            MSEs[idx2] += (j - y_test[idx][idx2])**2
    for idx3, _ in enumerate(MSEs):
        MSEs[idx3] = MSEs[idx3] / n
    return MSEs

def mae(y_pred, y_test):
    MAEs = [0, 0, 0]
    n = len(y_pred)
    for idx, i in enumerate(y_pred):
        for idx2, j in enumerate(i):
            MAEs[idx2] += abs(j - y_test[idx][idx2])
    for idx3, _ in enumerate(MAEs):
        MAEs[idx3] = MAEs[idx3] / n
    return MAEs

In [5]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42)
print(len(X_train), len(X_val), len(X_test))

21 4 2


In [6]:
#svm = SVR(kernel='linear', C=1.0)
#reg = MultiOutputRegressor(svm)
#reg.fit(X_train, y_train)

In [7]:
best_performance = 0
patience = 5  # Number of epochs with no improvement after which training will be stopped
no_improvement_count = 0
num_epochs = 100

svm = SVR(kernel='linear', C=10.0)
reg = MultiOutputRegressor(svm)

for epoch in range(num_epochs):
    # Train the SVM on the training data
    reg.fit(X_train, y_train)
    # Evaluate performance on the validation set
    y_pred = reg.predict(X_val)
    MSEs = mse(y_pred, y_val)
    current_performance = np.mean(MSEs)

    if current_performance > best_performance:
        print(f'New best MSE of {current_performance}')
        best_performance = current_performance
        no_improvement_count = 0
    else:
        no_improvement_count += 1

        if no_improvement_count >= patience:
            print(f'Early stopping at epoch {epoch}')
            break


New best MSE of 0.048854610978776136
Early stopping at epoch 5


In [8]:
y_pred = reg.predict(X_test)


for idx, i in enumerate(y_pred):
    print(f'IMAGE {idx+1}:')
    for idx2, j in enumerate(i):
        print(f'\t ROI{idx2+1}: real {y_test[idx][idx2]}, pred {j}.')
        
MSEs = mse(y_pred, y_test)
print(f'MSEs: {MSEs}')
print(f'Mean MSE: {np.mean(MSEs)}')

IMAGE 1:
	 ROI1: real 0.12540799442893116, pred 0.23468414624257578.
	 ROI2: real 0.095112566022786, pred 0.062318792898532246.
	 ROI3: real 0.026375528970517478, pred 0.021089422234651118.
IMAGE 2:
	 ROI1: real 0.027581862771305157, pred 0.1669832212019993.
	 ROI2: real 0.01077543959214951, pred 0.062318792898532246.
	 ROI3: real 0.0, pred 0.021089422234651118.
MSEs: [0.015687008043760782, 0.0018660744128958123, 0.00023635332730718433]
Mean MSE: 0.005929811927987926
